### GPT to LLAMA2
- GPT 모델의 코드를 수정해서 메타의 Llama 2 모델 구조로 개조하는게 목표
- Llama2를 선택한 이유는 Llama3와 구조가 거의 유사하기 때문

![gpt_vs_llama](https://camo.githubusercontent.com/fe670aeebfbcdb5b3d5c5a06fb720f4f8d599a21a85976c9b98a093aa2aa3794/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f626f6e75732f6770742d746f2d6c6c616d612f677074322d746f2d6c6c616d61322d6c6c616d61332e776562703f31)

- 이미지에서 볼 수 있듯이 거의 유사하지만 다른점이 조금씩 보임
    - LayerNorm 이 없어지고 RMSNorm
    - 활성화함수가 GELU 대신 SwiGLU
    - Embedding Dimension이 4096개.. (ㄷㄷ)
    - Position Embedding이 최대 4096 토큰 사이즈를 지원하면서 절대적 위치 뿐만 아니라 상대적 위치를 사용하는 RoPE 사용 가능
    - transformer 블럭이 32개로 줄어들음
    - 등등..

In [2]:
from importlib.metadata import version

pkgs = [
    "huggingface_hub",  # to download pretrained weights
    "sentencepiece",    # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

huggingface_hub version: 0.33.2
sentencepiece version: 0.2.0
torch version: 2.8.0


In [3]:
import torch
import torch.nn as nn


#####################################
# Chapter 4
#####################################

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift


class RMSNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.emb_dim = emb_dim
        self.weight = nn.Parameter(torch.ones(emb_dim)).float()

    def forward(self, x):
        
        # 1. 입력 벡터 원소 제곱의 평균 구하기
        means = x.pow(2).mean(dim=-1, keepdim=True)
        
        # 2. RMS로 입력 벡터 원소들 나누기
        # rsqrt 함수는 1/sqrt(x)와 동일  => 연산을 나눗셈에서 곱셈으로 바꾸기 위하여 
        x_normed = x * torch.rsqrt(means + self.eps)
        # 스케일링된 Input으로 Weight 연산
        return (x_normed * self.weight).to(dtype=x.dtype)

### LayerNorm to RMSNorm
- 왜 우린 Normalization을 할까?
    - Loss를 구하고 MSE를 계산하는 과정은 데이터간의 연산임
    - 데이터간 격차가 커질수록 Gradient가 커질수 밖에 없음 
    - 즉 안정적이고 효율적인 학습을 위해서는 데이터간의 스케일을 맞춰줄 필요가 있음
- LayerNorm
    - 입력 벡터의 데이터에서 평균을 빼고 분산으로 나눈다
    - 이 후 shift 연산을 통해 Bias를 더 함
- RMSNorm
    - LayerNorm 연산 과정에서 평균을 빼는 과정을 생략하고 입력 벡터의 크기로만 나눔
    - 벡터의 크기로만 나누기 때문에 벡터의 방향성은 유지되고 스케일링은 적용됨
    - llama2에서는 Bias 연산이 없기에 Shift 연산도 X
        - RMS 논문에서는 평균을 빼는 Centering 과정 없이도 스케일링 잘만 된다는걸 실험함
            - Root Mean Square Layer Normalization 2019

![layernorm_rmsnorm](https://towardsdatascience.com/wp-content/uploads/2024/09/07OWljs7EYEg2j-a0.png)

In [4]:
torch.manual_seed(123)

example_batch = torch.randn(2, 3, 4)

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
# Pytorch 내부적으로 RMSNorm은 구현되어 있음
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

# 예제에서 구현한 RMSNorm과 Pytorch의 RMSNorm이 같은지 검증
assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

In [5]:
#####################################
# Chapter 4
#####################################

# class GELU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         return 0.5 * x * (1 + torch.tanh(
#             torch.sqrt(torch.tensor(2.0 / torch.pi)) *
#             (x + 0.044715 * torch.pow(x, 3))
#         ))


class SiLU(nn.Module):
    def __init__(self):
        super(SiLU, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)
silu = SiLU()

assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

### GELU to SiLU
- Llama2에서는 활성화 함수가 GELU에서 SiLU로 변경되었다.

- GELU(Gaussian Error Linear Unit)
    - GPT-1 GPT-2 GPT-3 BERT 등 트랜스포머 모델들에서 자주 쓰이던 표준 활성화 함수
    - RELU와 달리 모든 구간이 미분가능하고 입력과 입력값의 정규분포 위치에 대한 확률을 곱한다
        - $\text{GELU}(x) = x \cdot \Phi(x)$
        - 해당 수식을 그대로 구현하려면 적분(정규분포의 누적분포함수)을 사용하기때문에 위 주석처리 되어있는 코드처럼 torch에서는 해당 수식을 근사한다.
        - $ \text{GELU}(x) = 0.5 x \left( 1 + \tanh \left[ \sqrt{\frac{2}{\pi}} \left( x + 0.044715 x^3 \right) \right] \right) $

![GELU_graph](https://docs.pytorch.org/docs/stable/_images/GELU.png)

- SiLU
    - GELU와 달리 수식 구현이 매우 간단하다.
        -$\text{SiLU}(x) = x \cdot \sigma(x) = x \cdot \frac{1}{1 + e^{-x}}$
    - 입력에 시그모이드 함수만 곱해준 걸로 활성화 함수를 구현한다.
    - 위 GELU 그래프와 SiLU 그래프를 비교해보면 형태가 매우 유사한 걸 알 수 있다.
    - 비슷한 형태임에도 GELU의 복잡한 연산을 단순화시켜 성능면에서 개선되어 Llama2에서 채택됨

![SiLU_graph](https://docs.pytorch.org/docs/stable/_images/SiLU.png)



In [6]:
#####################################
# Chapter 4
#####################################
# class FeedForward(nn.Module):
#     def __init__(self, cfg):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
#             GELU(),
#             nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
#         )

#     def forward(self, x):
#         return self.layers(x)
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        ## bias = False로 사용하지 않는걸 확인 할 수 있음
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)
        self.silu = SiLU()

    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        # 모든 Linear(입력데이터)가 활성화함수(SiLU)를 통과하는 것이 아닌 fc1만 통과후 fc2와 elementwise multiplication 함
        x = self.silu(x_fc1) * x_fc2
        # 이 결과를 fc3에 반영시켜 차원 유지
        return self.fc3(x)

- 기존 GPT2에서는 INPUT => Linear(차원 확장) => GELU(활성화함수) => Linear2 => 출력
    - 순차적으로 선형레이어 2개를 통과함
- Llama2에서는 GLU(Gates Linear Unit)라는 걸 도입함
    - Gate Linear(fc1) : 입력에 대해 활성화 함수를 계산
    - Value Linear(fc2) : 활성화함수 연산 없이 데이터만 가지고 있음
    - fc1과 fc2를 elementwise multiplication 하고 결과를 fc3에 반영
- 이게 가능한 이?유 (뇌피셜)
    - Input이 처음에 활성화 함수를 거치면서 Linear1이 잠재적으로 활성화함수 역할을 할 수 있다고 본듯??
    - Elementwise multiplication 하면서 fc1이 fc2의 유사 활성화함수 역할을 한다고 보는 느낌 ..

In [7]:
def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
    assert head_dim % 2 == 0, "Embedding dimension must be even"

    # theta_base 기준으로 차원이 커질수록 점점 느려지는 frequency를 생성
    # frequency를 만드는 이유는 head demension에 따라 고유한 ㅇ
    inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[: (head_dim // 2)].float() / head_dim))

    # 위치 좌표 (0,1,2...,4095)
    positions = torch.arange(context_length)

    # 각도(radian) 계산 (위치와 frequency를 곱해 위치에 따른 회전 각도를 미리 계산함)
    angles = positions.unsqueeze(1) * inv_freq.unsqueeze(0)  # Shape: (context_length, head_dim // 2)

    # 삼각함수(sin,cos) 계산을 위해 차원 맞춰주기 angles=[theta1,theta2]면 [theta1,theta2,theta1,theta2]
    angles = torch.cat([angles, angles], dim=1)  # Shape: (context_length, head_dim)

    # 각도에 따른 Sin Cos를 미리 구해놓기
    cos = torch.cos(angles)
    sin = torch.sin(angles)

    return cos, sin

def compute_rope(x, cos, sin):
    # x: (batch_size, num_heads : 어텐션 헤드 개수, seq_len : 문장 길이, head_dim : 단어 차원크기)
    batch_size, num_heads, seq_len, head_dim = x.shape
    assert head_dim % 2 == 0, "Head dimension must be even"

    # 입력벡터 x의 차원을 반갈죽낸다.
    # 현재 x1은 [x1,x2,...x31]이 들어있고 x2는 [x32,x33,x34...x63]이 들어있게 된다 ..
    # 나중에 이 둘이 짝지어 회전하게 된다. x0과 x32 / x1와 x33 ..
    # 이렇게 짝지어서 하는 이유는 나중에 회전행렬의 계산을 편하게 하기 위해서 !
    x1 = x[..., : head_dim // 2]
    x2 = x[..., head_dim // 2 :]

    # 미리 계산해둔 cos값 sin값을 문장 길이에 맞춰 자른다.
    # 입력의 차원에 맞춰준다. (문장길이, 단어 자체의 차원 크기)
    cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, seq_len, head_dim)
    sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

    # 회전 행렬의 공식을 적용한 부분임..
    rotated = torch.cat((-x2, x1), dim=-1)
    x_rotated = (x * cos) + (rotated * sin)

    return x_rotated.to(dtype=x.dtype)

# Settings
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

# Instantiate RoPE parameters
cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

# Dummy query and key tensors
torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)

# Apply rotary position embeddings
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

### Absolute Position Embedding to RoPE
- GPT에서는 단어 벡터에 위치 정보를 담은 벡터를 단순히 더함
    ```python
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    ```
    - Position Embedding은 단순히 단어의 index를 나타내며 단어 index간의 거리에 대한 정보는 없음
        - 예를들면 문장이 길면 앞에서 나온 단어와 뒤에서 나온 단어의 관련성이 줄어들지만 이에 대한 정보X
            - 문장이 길어질수록 성능이 안좋아짐
- RoPE는 상대적인 위치를 표현하기 위해 단어벡터를 일정한 각도로 회전시킴
    - 왜 굳이 이런짓을 ..
        - 직관적으로 Relatve Position Embedding을 구현하기 위해서는 토큰*토큰 size의 matrix를 만들고 상대적인 거리를 표현하면됨
        - 메모리 소모도 커지고 연산량도 매우 커짐
        - RoPE는 이를 해결하기 위해 토큰의 상대적인 거리를 회전좌표계로 표현
    - RoPE 회전 좌표계 구하기
    - 
        ```python     
                rotated = torch.cat((-x2, x1), dim=-1)
                x_rotated = (x * cos) + (rotated * sin)
        ```
        - 위 코드는 회전 행렬 계산을 구현한 것
            - 반갈죽 했던 이유가 여기에 있음 x 와 rotated로 간단히 구현함
            - [rotation_matrix](https://wikimedia.org/api/rest_v1/media/math/render/svg/e02da33f45679713d15de997449a76df48efb282)

    - $R_{\theta, p} \begin{bmatrix} x_{2i} \\ x_{2i+1} \end{bmatrix} = \begin{bmatrix} \cos(p\theta) & -\sin(p\theta) \\ \sin(p\theta) & \cos(p\theta) \end{bmatrix} \begin{bmatrix} x_{2i} \\ x_{2i+1} \end{bmatrix}$

In [8]:
#####################################
# Chapter 3
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):  # ,dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        ################################### NEW ###################################
        # Set bias=False and dtype=dtype for all linear layers below
        ###########################################################################
        
        # 늘 있는 QKV 
        self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
        
        # 어텐션 계산 끝난후 결과를 하나로 합치는 Linear
        self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # Linear layer to combine head outputs
        # self.dropout = nn.Dropout(dropout)
        
        # 트랜스포머에서 흔히 봤던 컨닝방지용 mask
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        ################################### NEW ###################################
        cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_length)
        
        ## 미리 계산해둔 cosine sin 값을 register_buffer를 통해 저장해둠
        self.register_buffer("cos", cos)
        self.register_buffer("sin", sin)
        ###########################################################################


    def forward(self, x):

        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 멀티 헤드 어텐션을 위한 쪼개기 (b,10,768) -> (b,10,12,64)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # 이 후 회전좌표계 계산할때 num_tokens, head_dim을 기준으로 계산하기때문에 transpose
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        ################################### NEW ###################################
        # keys, queries 벡터를 회전시킴
        keys = compute_rope(keys, self.cos, self.sin)
        queries = compute_rope(queries, self.cos, self.sin)
        ###########################################################################

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        # (b,12,10,64) matrix multiplicataion (b,12,64,10) => (b,12,10,10)
        # (10,10) Score Matrix를 얻을 수 있음
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # 늘 있는 마스킹
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        # 늘 있는 QKV 연산
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        # 헤드 합치고 마무리
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec
    
# Settings
batch_size = 1
context_len = 100
max_context_len = 4096
embed_dim = 128
num_heads = 4


example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
    d_in=embed_dim,
    d_out=embed_dim,
    context_length=max_context_len,
    num_heads=num_heads
)

mha(example_batch)

del mha  # delete to free up memory

- 기존 MHA 연산에서 Positional Embedding이 회전좌표계 일 경우의 코드 구현
    - (토큰 개수(문장길이), 헤드 차원)을 기준으로 회전 좌표계를 구하기 위해서 Transpose 한 걸 제외하고는 비슷한듯함 ..

In [9]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dtype=cfg["dtype"]  # NEW
            # dropout=cfg["drop_rate"],
            # qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)

        ################################### NEW ###################################
        # self.norm1 = LayerNorm(cfg["emb_dim"])
        # self.norm2 = LayerNorm(cfg["emb_dim"])
        self.norm1 = RMSNorm(cfg["emb_dim"])
        self.norm2 = RMSNorm(cfg["emb_dim"])
        ###########################################################################

        # self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        # x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

### Transformer Block
- GPT에서 사용했던 Transformer block에서 바뀐점은
    - dropout, qkv_bias를 사용하지 않음
    - Layernorm에서 RMSNorm
    - Activation Function을 적용 받지 않는 Linear도 존재
    - dtype 추가 => 데이터 로드 시 메모리에서 dfloat16 16비트 사용하게 해서 메모리 절약
    - Resiudal Connection으로 구현되어 있는것 같다 ..
        - shortcut으로 원본을 담아 둔 후 정규화 => 어텐션연산/활성화함수 => 원본 과 결과 더하고 출력

In [ ]:
# class GPTModel(nn.Module):
class Llama2Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
        # self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        ################################### NEW ###################################
        # self.final_norm = LayerNorm(cfg["emb_dim"])
        self.final_norm = RMSNorm(cfg["emb_dim"])
        ###########################################################################
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=cfg["dtype"])

    def forward(self, in_idx):
        # batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        # pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds  # + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        # x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

### Model Class
- GPT에서 사용했던 Model Class와 비교했을때 Absolute Positional Embedding 부분이 날아갔다 ..

In [ ]:
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # Vocabulary size
    "context_length": 4096,  # Context length
    "emb_dim": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "hidden_dim": 11008,     # NEW: Size of the intermediate dimension in FeedForward
    "dtype": torch.bfloat16  # NEW: Lower-precision dtype to reduce memory usage
}

model = Llama2Model(LLAMA2_CONFIG_7B)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

def model_memory_size(model, input_dtype=torch.float32):
    total_params = 0
    total_grads = 0
    for param in model.parameters():
        # Calculate total number of elements per parameter
        param_size = param.numel()
        total_params += param_size
        # Check if gradients are stored for this parameter
        if param.requires_grad:
            total_grads += param_size

    # Calculate buffer size (non-parameters that require memory)
    total_buffers = sum(buf.numel() for buf in model.buffers())

    # Size in bytes = (Number of elements) * (Size of each element in bytes)
    # We assume parameters and gradients are stored in the same type as input dtype
    element_size = torch.tensor(0, dtype=input_dtype).element_size()
    total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

    # Convert bytes to gigabytes
    total_memory_gb = total_memory_bytes / (1024**3)

    return total_memory_gb

print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")

- 무지무지막지한 파라미터와 메모리를 보라 ..

In [ ]:
from huggingface_hub import login,hf_hub_download
import json
import sentencepiece as spm

with open("config.json", "r") as config_file:
    config = json.load(config_file)
    access_token = config["HF_ACCESS_TOKEN"]

login(token=access_token)

tokenizer_file = hf_hub_download(
    repo_id="meta-llama/Llama-2-7b",
    filename="tokenizer.model",
    local_dir="Llama-2-7b"
)

class LlamaTokenizer:
    def __init__(self, tokenizer_file):
        sp = spm.SentencePieceProcessor()
        sp.load(tokenizer_file)
        self.tokenizer = sp

    def encode(self, text):
        return self.tokenizer.encode(text, out_type=int)

    def decode(self, ids):
        return self.tokenizer.decode(ids)


tokenizer = LlamaTokenizer(tokenizer_file)



- 액세스 토큰 연결하는건 깃허브 레포 참조

In [ ]:
weights_file = hf_hub_download(
   repo_id="meta-llama/Llama-2-7b",
   filename="consolidated.00.pth",
   local_dir="Llama-2-7b"
)
weights = torch.load(weights_file, weights_only=True)
list(weights.keys())[:15]

- 이후 Llama 모델과 연결하고 가중치를 불러오는 예제가 진행되는데 ..
- 노트북이 Llama2를 못버팀 ..ㅜㅜㅜㅜㅜ
    - GPT2와 매우 유사하므로 생략하겟슴 ..